# AI Red Teaming Agent

This code sample enables creating an AI Red Teaming Agent using Azure AI Evaluation and Semantic Kernel.

## Overview

The `template.py` script implements an AI Red Teaming Agent that can:

1. Fetch harmful prompts across different [risk categories](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent#supported-risk-categories)
2. Send prompts to target generative AI models or applications
3. Apply transformations to prompts (like base64 encoding) with different [attack strategies](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent#supported-attack-strategies)
4. Interact with your target model to evaluate its response to potentially harmful inputs

## Prerequisites

- Python 3.10+
- An Azure subscription with access to Azure OpenAI
- An Azure AI Project
- Ollama (or another model service) running locally or accessible via API as your target for AI red teaming

## Setup Instructions

### 1. Install Required Dependencies

In [ ]:
# !pip install semantic-kernel azure-ai-evaluation[redteam] python-dotenv requests

### 2. Environment Variables

Create a `.env` file in the same directory as `template.py` with the following variables:

```
# Azure OpenAI Configuration
AZURE_PJ_MODEL_ENDPOINT=https://your-resource-name.openai.azure.com/
AZURE_PJ_MODEL_NAME=your-deployment-name
AZURE_PJ_MODEL_API_KEY=your-api-key

# Azure AI Project Endpoint, get it on Azure AI Froundry UI project home page
AZURE_PJ_PROJECT_ENDPOINT=https://your-resource-name.services.ai.azure.com/api/projects/your-project-name
```

### 3. Configure Target Model

By default, the template uses Ollama as the target model. To use Ollama:

1. Make sure Ollama is installed and running on your machine
2. Update the model name in the `call_ollama` function:

In [ ]:
payload = {"model": "llama2", "prompt": query, "stream": False}  # Replace "llama2" with your model

### 4. Changing the Target Model

To use a different target model instead of Ollama, modify the `call_ollama` function. For example, to use another API:

In [ ]:
def call_custom_api(query: str) -> str:
    """
    Call a custom API with a prompt and return the response.
    """
    url = "https://your-api-endpoint.com/generate"
    headers = {
        "Authorization": f"Bearer {os.environ.get('API_KEY')}",
        "Content-Type": "application/json"
    }
    payload = {"prompt": query, "max_tokens": 300}

    response = requests.post(url, headers=headers, json=payload, timeout=60)
    try:
        return response.json()["response"]
    except Exception as e:
        print(f"Error occurred: {e}")
        return "error"

Then, update the `main` function to use your new target function:

In [ ]:
# Initialize the RedTeamPlugin with the target function
red_team_plugin = RedTeamPlugin(
    AZURE_PJ_PROJECT_ENDPOINT=ai_project_endpoint,
    target_func=call_custom_api  # Use your new function here
)

## Running the Agent

Execute the script:

In [ ]:
!python template.py

The script will:
1. Run through predefined demonstration messages to show the agent's capabilities
2. Enter interactive mode where you can interact with the agent directly

## Usage Examples

Here are some example commands you can use in interactive mode:

- `Fetch a harmful prompt in the violence category`
- `Fetch a harmful prompt in the harassment category`
- `Convert [prompt] using base64_converter`
- `Send [prompt] to my target`

## Customizing Agent Behavior

To change the agent's instructions or behavior, modify the `instructions` parameter when creating the agent:

In [ ]:
agent = ChatCompletionAgent(
    service=service,
    name="RedTeamAgent",
    instructions="Your custom instructions here...",
    plugins=[red_team_plugin],
)

## Security and Responsible Use

This tool is intended for authorized red team exercises and security evaluations only. Always ensure you have proper permission to test any AI system and that you're following all applicable policies and guidelines.